In [460]:
import json
import os

In [461]:
from nltk.sentiment import SentimentIntensityAnalyzer

In [462]:
minLikes = 10000000
maxLikes = 0

minReplies = 10000000
maxReplies = 0

minRetweets = 10000000
maxRetweets = 0

In [463]:
# Utility Method To Find All Politicians Whose Tweets Are Available

def get_politicians():
    path = "tweets-test"
    folders = [folder for folder in os.listdir(path) if os.path.isdir(os.path.join(path, folder))]

    for folder in folders:
        yield folder

In [464]:
# Utility Method To Find All The Available Tweets From A Given Politician

def get_politicians_tweets(politician):
    global maxLikes
    global minLikes

    global maxReplies
    global minReplies

    global maxRetweets
    global minRetweets

    path = f"tweets-test/{politician}"
    tweets = []

    for filename in os.listdir(path):
        file_path = os.path.join(path, filename)

        if os.path.isfile(file_path):
            with open(file_path, 'r', encoding="utf-8") as file:
                data = json.load(file)

                if data["Likes"] > maxLikes:
                    maxLikes = data["Likes"]
                elif data["Likes"] < minLikes:
                    minLikes = data["Likes"]

                if data["Replies"] > maxReplies:
                    maxReplies = data["Replies"]
                elif data["Replies"] < minReplies:
                    minReplies = data["Replies"]

                if data["Retweets"] > maxRetweets:
                    maxRetweets = data["Retweets"]
                elif data["Retweets"] < minRetweets:
                    minRetweets = data["Retweets"]

                tweets.append(data)

    return tweets

In [465]:
def sentiment_polarity(text):
    sia = SentimentIntensityAnalyzer()

    score = sia.polarity_scores(text)
    key = list(score.keys())[list(score.values()).index(max(list(score.values())[:len(score) - 1]))]

    return score, key

In [466]:
def calculate_score(json_tweet):
    score, key = sentiment_polarity(json_tweet["Tweet"])
    # The Sentiment Polarity Is Worth 25% Of The Total Score
    # The Sentiment Polarity Also Cannot Be Negative
    sent_pol = (score["pos"] - score["neg"]) if (score["pos"] - score["neg"]) > 0 else 0

    likes_score = (json_tweet["Likes"] - minLikes) / (maxLikes - minLikes)  # Scales The Likes Score To The Range Of 0.0 to 1.0
    replies_score = (json_tweet["Replies"] - minReplies) / (maxReplies - minReplies)  # Scales The Replies Score To The Range Of 0.0 to 1.0
    retweets_score = (json_tweet["Retweets"] - minRetweets) / (maxRetweets - minRetweets)  # Scales The Retweets Score To The Range Of 0.0 to 1.0

    total_score = 25 * (sent_pol + likes_score + replies_score + retweets_score)  # The Final Total Score Is From 0.0 to 100.0
    return total_score

In [467]:
def analyze_politician(politician):
    total_score = 0
    tweet_count = 0
    all_politicians_tweets = get_politicians_tweets(politician)

    for tweet in all_politicians_tweets:
        total_score += calculate_score(tweet)
        tweet_count += 1

    avg_score = total_score / tweet_count

    return avg_score

In [468]:
def store_results(results):
    with open("./results.json", "w+") as file:
        json.dump(results, file)

In [469]:
def analyze():
    scores = {}
    all_politicians = get_politicians()

    while True:
        try:
            politician = next(all_politicians)
            scores.update({ politician: analyze_politician(politician) })

        except StopIteration:
            break

    store_results(scores)

In [470]:
def conclude():
    pass

In [471]:
def main():
    analyze()
    conclude()


if __name__ == '__main__':
    main()